# NC adatelemzés

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
anomaly_num = 0

In [58]:
df = pd.read_csv("PLC_NBH170.csv", sep=";")

In [59]:
def switch(i):
    switcher = {
        'KBBUD10449-NBH170-PLCStationThing': "10449",
        'KBBUD10704-NBH170-PLCStationThing': "10704",
        'KBBUD10469-NBH170-PLCStationThing': "10469",
        'KBBUD10460-NBH170-PLCStationThing': "10460",
        'KBBUD10395-NBH170-PLCStationThing': "10395",
        'KBBUD10424-NBH170-PLCStationThing': "10424",
        'KBBUD10419-NBH170-PLCStationThing': "10419",
        'KBBUD10396-NBH170-PLCStationThing': "10396"
        
    }
    return switcher.get(i,"invalid")

In [60]:
df['Machine'] = df['Machine'].apply(lambda x: switch(x))
df['StatusTime'] = df['StatusTime'].apply(lambda x: x[:-4])

del df['StatusType']
del df['description']
del df['CreationTime']

# NC Státuszok

In [61]:
df = df[df['Status']=='NC']
del df['Status']
df = df.sort_values(by='StatusTime')

In [62]:
df

,Id,Machine,StatusTime,SubStatus
29,115,10395,2019-08-21 13:24:50,STOP
45,145,10460,2019-08-21 13:25:49,START
50,156,10460,2019-08-21 13:26:09,STOP
82,254,10395,2019-08-21 13:30:04,START
93,280,10395,2019-08-21 13:32:02,STOP
...,...,...,...,...
615021,2012959,10395,2019-11-19 13:09:43,STOP
615025,2012981,10395,2019-11-19 13:11:03,START
615040,2013109,10469,2019-11-19 13:16:41,STOP
615042,2013113,10419,2019-11-19 13:16:46,START


anomálisa mentesítés gépenként

In [63]:
list = []
list.append(df[df['Machine']=='10449'])
list.append(df[df['Machine']=='10704'])
list.append(df[df['Machine']=='10469'])
list.append(df[df['Machine']=='10460'])
list.append(df[df['Machine']=='10395'])
list.append(df[df['Machine']=='10424'])
list.append(df[df['Machine']=='10419'])
list.append(df[df['Machine']=='10396'])

In [64]:
# Anomália kezelés 1: több START jön egymás után, akkor az ELSŐ érvényes, több STOP jön egymás után, az utolsó az érvényes

def anomaly_handle_1(list):
    global anomaly_num
    anomaly_num = 1

    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'START' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'START'):     
                    list[j]['SubStatus'].iloc[i+1] = 'Anomaly'
            elif(list[j]['SubStatus'].iloc[i] == 'Anomaly'):
                if(list[j]['SubStatus'].iloc[i+1] == 'START'):     
                    list[j]['SubStatus'].iloc[i+1] = 'Anomaly'
    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'STOP' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'STOP'):     
                    list[j]['SubStatus'].iloc[i] = 'Anomaly'

In [65]:
# Anomália kezelés 2: több START jön egymás után, akkor az UTOLSÓ érvényes, több STOP jön egymás után, az utolsó az érvényes

def anomaly_handle_2(list):
    global anomaly_num
    anomaly_num = 2

    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'START' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'START'):     
                    list[j]['SubStatus'].iloc[i] = 'Anomaly'
    for j in range(len(list)):
        for i in range(len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'STOP' ):
                if(list[j]['SubStatus'].iloc[i+1] == 'STOP'):     
                    list[j]['SubStatus'].iloc[i] = 'Anomaly'

In [66]:
anomaly_handle_2(list)

In [67]:
j = 0
k = 0
for i in range(len(list[0])):
    if(list[0]['SubStatus'].iloc[i] == 'START'):
        j = j + 1
    elif(list[0]['SubStatus'].iloc[i] == 'STOP'):
        k = k + 1

print("START: " + str(j))
print("STOP: " + str(k))

START: 1548
STOP: 1548


In [68]:
list_anomaly = []

for i in range(len(list)): 
    list_anomaly.insert(i, list[i]) 

In [69]:
for j in range(len(list)):
    list[j] = list[j][list[j]['SubStatus']!='Anomaly']

# idő hozzárendelése gépenként

In [70]:
# startal kell hogy kezdődjön és stoppal végződnie

In [71]:
for i in range(len(list_anomaly)):
    if(list_anomaly[i].iloc[0]['SubStatus'] != 'START'):
        list_anomaly[i] = list_anomaly[i].drop([list_anomaly[i].index[0]])
    if(list_anomaly[i].iloc[len(list_anomaly[i]) - 1]['SubStatus'] != 'STOP'):
        list_anomaly[i] = list_anomaly[i].drop([list_anomaly[i].index[len(list_anomaly[i])-1]])
    if(list_anomaly[i].iloc[len(list_anomaly[i]) - 1]['SubStatus'] == 'Anomaly'):
        list_anomaly[i] = list_anomaly[i].drop([list_anomaly[i].index[len(list_anomaly[i])-1]])

for i in range(len(list)):
    if(list[i].iloc[0]['SubStatus'] != 'START'):
        list[i] = list[i].drop([list[i].index[0]])
    if(list[i].iloc[len(list[i]) - 1]['SubStatus'] != 'STOP'):
        list[i] = list[i].drop([list[i].index[len(list[i])-1]])

In [72]:
from datetime import datetime, timedelta

In [73]:
def timeInWorking(list): 
    for j in range(len(list)):    
        list[j]['timeInWorking'] = timedelta(0)
        for i in range(len(list[j])-1)[0::2]:
            timeACTIVE_Str = list[j]['StatusTime'].iloc[i]
            timeEND_Str = list[j]['StatusTime'].iloc[i+1]
            timeStart = datetime.strptime(timeACTIVE_Str,'%Y-%m-%d %H:%M:%S')
            timeEND = datetime.strptime(timeEND_Str,'%Y-%m-%d %H:%M:%S')
            list[j]['timeInWorking'].iloc[i+1] = timeEND-timeStart

In [74]:
def timeNotWorking(list): 
    for j in range(len(list)):
        list[j]['timeNotWorking'] = timedelta(0)
        for i in range(len(list[j])-1)[1::2]:
            timeACTIVE_Str = list[j]['StatusTime'].iloc[i]
            timeEND_Str = list[j]['StatusTime'].iloc[i+1]
            timeStart = datetime.strptime(timeACTIVE_Str,'%Y-%m-%d %H:%M:%S')
            timeEND = datetime.strptime(timeEND_Str,'%Y-%m-%d %H:%M:%S')
            list[j]['timeNotWorking'].iloc[i+1] = timeEND-timeStart

In [75]:
def timeInAnomaly(list):
    for j in range(len(list)):
        list[j]['timeInAnomaly'] = timedelta(0)

        i = 0
        while(i <= len(list[j])-1):
            if(list[j]['SubStatus'].iloc[i] == 'Anomaly' ):
                anomalyStart_Str = list[j]['StatusTime'].iloc[i]
                #ameddig anomália jön, addig megyünk
                k = 1
                while list[j]['SubStatus'].iloc[k + i] == 'Anomaly':
                    k = k + 1

                if(k == 1):
                    i += 1
                    pass
                else:
                    anomalyEND_Str = list[j]['StatusTime'].iloc[i + k]
                    timeStart = datetime.strptime(anomalyStart_Str,'%Y-%m-%d %H:%M:%S')
                    timeEND = datetime.strptime(anomalyEND_Str,'%Y-%m-%d %H:%M:%S')
                    list[j]['timeInAnomaly'].iloc[i + k] = timeEND-timeStart  
                    i = i + k
                    k = 1
            else:
                i += 1 

In [76]:
timeInWorking(list)
timeNotWorking(list)

In [77]:
timeInAnomaly(list_anomaly)

In [78]:
print('nc in working time','   ','nc when not working','    ','all time sum of the machine')
print('')
for i in range(len(list)):
    print(list[i]['timeInWorking'].sum(),'     ', list[i]['timeNotWorking'].sum(),'       ',list[i]['timeInWorking'].sum()+list[i]['timeNotWorking'].sum())

nc in working time     nc when not working      all time sum of the machine

56 days 22:57:17       32 days 18:38:23         89 days 17:35:40
54 days 09:18:37       35 days 06:23:03         89 days 15:41:40
41 days 20:49:31       48 days 01:46:48         89 days 22:36:19
27 days 17:12:20       26 days 15:54:57         54 days 09:07:17
33 days 09:41:51       56 days 13:57:48         89 days 23:39:39
43 days 23:57:29       45 days 16:34:31         89 days 16:32:00
38 days 00:46:06       51 days 18:51:09         89 days 19:37:15
56 days 06:53:19       33 days 15:22:54         89 days 22:16:13


In [79]:
for i in range(len(list_anomaly)):
    print(str(i+1) + " számú gép: ",list_anomaly[i]['timeInAnomaly'].sum())

1 számú gép:  3 days 13:00:12
2 számú gép:  9 days 13:27:40
3 számú gép:  1 days 23:36:12
4 számú gép:  5 days 03:10:53
5 számú gép:  18 days 12:05:44
6 számú gép:  10 days 18:42:49
7 számú gép:  14 days 13:44:46
8 számú gép:  10 days 13:19:48


In [80]:
for i in range(len(list_anomaly)-1):
    del list_anomaly[i]['Machine']
    del list_anomaly[i]['StatusTime']
    del list_anomaly[i]['SubStatus']

In [81]:
list_new = []
for i in range(len(list)-1):
    list_new.append(pd.merge(list[i], list_anomaly[i], left_on='Id', right_on='Id', how='left'))

In [82]:
df = pd.concat(list_new)

df.to_csv('nc_status_2.csv', sep=";")

In [83]:
df

,Id,Machine,StatusTime,SubStatus,timeInWorking,timeNotWorking,timeInAnomaly
0,1612,10449,2019-08-21 14:57:59,START,00:00:00,00:00:00,0 days
1,1859,10449,2019-08-21 15:13:03,STOP,00:15:04,00:00:00,0 days
2,1976,10449,2019-08-21 15:17:38,START,00:00:00,00:04:35,0 days
3,2156,10449,2019-08-21 15:25:49,STOP,00:08:11,00:00:00,0 days
4,2211,10449,2019-08-21 15:27:47,START,00:00:00,00:01:58,0 days
...,...,...,...,...,...,...,...
3531,2009713,10419,2019-11-19 09:24:15,STOP,00:09:38,00:00:00,0 days
3532,2009739,10419,2019-11-19 09:25:33,START,00:00:00,00:01:18,0 days
3533,2009749,10419,2019-11-19 09:26:51,STOP,00:01:18,00:00:00,0 days
3534,2010060,10419,2019-11-19 09:37:54,START,00:00:00,00:11:03,0 days


In [84]:
import os
from datetime import datetime as dt
import datetime
import pytz, time
import gc

os.environ['TZ'] = 'Europe/Budpest'
CET = pytz.timezone('Europe/Budapest')

In [85]:
df = df[['Id', 'Machine', 'StatusTime', 'SubStatus', 'timeInWorking', 'timeNotWorking', 'timeInAnomaly']]
df = df.astype({'StatusTime' : 'datetime64'})
df.index = df.StatusTime
df.index = df.index.tz_localize('Europe/Budapest')
df = df.drop(columns='StatusTime')
df.sort_values(inplace=True, axis='index', by='StatusTime')
df.head()

,Id,Machine,SubStatus,timeInWorking,timeNotWorking,timeInAnomaly
StatusTime,,,,,,
2019-08-21 13:25:49+02:00,145,10460,START,00:00:00,00:00:00,0 days
2019-08-21 13:26:09+02:00,156,10460,STOP,00:00:20,00:00:00,0 days
2019-08-21 13:30:04+02:00,254,10395,START,00:00:00,00:00:00,0 days
2019-08-21 13:32:02+02:00,280,10395,STOP,00:01:58,00:00:00,0 days
2019-08-21 13:57:53+02:00,644,10395,START,00:00:00,00:25:51,0 days


In [86]:
df['ts_work'] = 0
df['ts_notwork'] = 0
df['ts_anom'] = 0
for i in range(len(df)):
    df['ts_work'].iloc[i] = df['timeInWorking'].iloc[i].total_seconds() / 3600
    df['ts_notwork'].iloc[i] = df['timeNotWorking'].iloc[i].total_seconds() / 3600
    df['ts_anom'].iloc[i] = df['timeInAnomaly'].iloc[i].total_seconds() / 3600

In [87]:
del df['timeInWorking']
del df['timeNotWorking']
del df['timeInAnomaly']
df

,Id,Machine,SubStatus,ts_work,ts_notwork,ts_anom
StatusTime,,,,,,
2019-08-21 13:25:49+02:00,145,10460,START,0.000000,0.000000,0.0
2019-08-21 13:26:09+02:00,156,10460,STOP,0.005556,0.000000,0.0
2019-08-21 13:30:04+02:00,254,10395,START,0.000000,0.000000,0.0
2019-08-21 13:32:02+02:00,280,10395,STOP,0.032778,0.000000,0.0
2019-08-21 13:57:53+02:00,644,10395,START,0.000000,0.430833,0.0
...,...,...,...,...,...,...
2019-11-19 13:06:22+01:00,2012853,10395,START,0.000000,0.005556,0.0
2019-11-19 13:08:02+01:00,2012911,10395,STOP,0.027778,0.000000,0.0
2019-11-19 13:09:23+01:00,2012941,10395,START,0.000000,0.022500,0.0


In [88]:
import influxdb

client = influxdb.DataFrameClient(host='152.66.34.82', port=8086)
client.switch_database('knorr')
client.query(query='DROP MEASUREMENT "nc_status"', database='knorr')
client.write_points(df, database='knorr', measurement='nc_status', tag_columns=['Machine', 'SubStatus'], field_columns=['ts_work','ts_notwork','ts_anom'], time_precision='ms', batch_size=10000, protocol='line')
client.query(query='SELECT * FROM nc_status', database='knorr')

defaultdict(list,
            {'nc_status':                           Machine SubStatus  ts_anom  ts_notwork   ts_work
             2019-08-21 11:25:49+00:00   10460     START      0.0    0.000000  0.000000
             2019-08-21 11:26:09+00:00   10460      STOP      0.0    0.000000  0.005556
             2019-08-21 11:30:04+00:00   10395     START      0.0    0.000000  0.000000
             2019-08-21 11:32:02+00:00   10395      STOP      0.0    0.000000  0.032778
             2019-08-21 11:57:53+00:00   10395     START      0.0    0.430833  0.000000
             ...                           ...       ...      ...         ...       ...
             2019-11-19 12:06:22+00:00   10395     START      0.0    0.005556  0.000000
             2019-11-19 12:08:02+00:00   10395      STOP      0.0    0.000000  0.027778
             2019-11-19 12:09:23+00:00   10395     START      0.0    0.022500  0.000000
             2019-11-19 12:09:43+00:00   10395      STOP      0.0    0.000000  0.005556
 